In [1]:
#importing generic modules
import csv
import textacy
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import pickle
import math

In [2]:
TRAIN = False #set true to train all models
EVAL = not TRAIN
ModelList = []

*DATA LOADING AND PRE-PROCESSING*

In [3]:
#reading data
data = []
with open('/home/aniket/Subtask-A/V1.4_Training.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    pos = 0
    neg = 0
    for i, row in enumerate(csv_reader):
        if i == 0:
            continue
        data.append({})
        data[-1]['paper_id'] = row[0]
        data[-1]['text'] = row[1]
        data[-1]['label'] = row[2]
        if int(data[-1]['label']) == 1:
            pos+= 1
        else:
            neg+= 1

In [8]:
#preprocessing data
def preprocess(data):
    for instance in data:
        text = instance['text']
        text = textacy.preprocess_text(text, fix_unicode=True,
                                            lowercase=True,
                                            no_urls=True,
                                            no_emails=True,
                                            no_phone_numbers=True,
                                            no_numbers=True,
                                            no_currency_symbols=True,
                                            no_punct=False,
                                            no_contractions=True,
                                            no_accents=True)
        text = text.replace('\n', ' ').strip('\'').strip('"')
        text = text.replace('___', ' ')
        text = text.replace('-', ' ')
        text = text.replace('__', ' ')
        text = text.replace('""""""', '"')
        text = text.replace('"""""', '"')
        text = text.replace('""""', '"')
        text = text.replace('"""', '"')
        text = text.replace(',', '')
        instance['text'] = text
        instance['tokenized_text'] = word_tokenize(text)
    return data

In [9]:
#validation data
val_data = []
with open('/home/aniket/Subtask-A/SubtaskA_Trial_Test_Labeled.csv', encoding = 'ISO-8859-1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for i, row in enumerate(csv_reader):
        if i == 0:
            continue
        val_data.append({})
        val_data[-1]['paper_id'] = row[0]
        val_data[-1]['text'] = row[1]
        val_data[-1]['label'] = row[2]

In [10]:
#final test data
test_data = []
test_data_csv = []
with open('/home/aniket/Subtask-A/SubtaskA_EvaluationData.csv', encoding = 'ISO-8859-1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for i, row in enumerate(csv_reader):
        test_data.append({})
        test_data_csv.append([row[0], row[1], row[2]])
        test_data[-1]['paper_id'] = row[0]
        test_data[-1]['text'] = row[1]
        test_data[-1]['label'] = row[2]

In [11]:
#driver for preprocessing validation and train data 
from random import shuffle
data = preprocess(data)
val_data = preprocess(val_data)
test = preprocess(test_data)
shuffle(data)

**Deep Learning**

My proposed approach is to use a pretrained model. I use BERT-base(12 layers and 768 features). But, only using a pretrained model yeilds an f1-score of about 0.75 on validation data. Henceforth, all f1-scores reprted will be on validation data. I implemented some models in which the BERT was finetuned and some in which the BERT weights are freezed. I added other architectural units (LSTM, CNN) to BERT and the best f1-score I observed was 0.84-0.85. The approach of model-3 gives this score.

In [12]:
#!pip install pytorch-pretrained-bert 

In [13]:
#install ml specific modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer, BertModel
from pytorch_pretrained_bert.optimization import BertAdam
from sklearn.metrics import precision_score, recall_score
import numpy as np

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [14]:
#batch data
def batchify(data, batch_size = 16):
    batched_text = []
    batched_labels = []
    cur_batch_text = []
    cur_batch_labels = []
    num_done = 0
    for i, d in enumerate(data):
        if i % batch_size == 0 and i != 0:
            batched_text.append(cur_batch_text)
            batched_labels.append(cur_batch_labels)
            cur_batch_labels = []
            cur_batch_text = []
            num_done = i
        cur_batch_text.append(d['text'])
        try:
            cur_batch_labels.append(int(d['label']))
        except:
            cur_batch_labels.append(-1)
            
    if num_done < len(data):
        batched_text.append(cur_batch_text)
        batched_labels.append(cur_batch_labels)
    return batched_text, batched_labels

In [15]:
#training loop for all models
def train(model, optimizer, data, val_data, out_file):
    #getting data as batches
    batched_text, batched_labels = batchify(data)
    val_batched_text, val_batched_labels = batchify(val_data)
    performance = []
    #training
    for epoch in range(30):
        train_loss = 0
        train_preds = []
        train_targets = []
        performance.append({})
        #looping through the training set
        for i in tqdm(range(len(batched_labels))):
            x = batched_text[i]
            y = batched_labels[i]
            preds = model(x)
            loss = model.loss(preds, y)
            model.zero_grad()
            loss.backward()
            optimizer.step()
            train_preds.append(preds)
            train_targets.append(y)
            train_loss = train_loss + loss.data
        train_preds = [k[i] for k in train_preds for i in range(k.shape[0])]
        train_targets = [i for item in train_targets for i in item]
        eval = model.eval(train_preds, train_targets)
        train_loss = train_loss / len(batched_text)
        print('epoch:'+str(epoch)+' loss:'+str(train_loss)+' precision:'+str(eval[0])+
              ' recall:'+str(eval[1])+' f1:'+str(eval[2]))

        test_preds = []
        test_targets = []
        #looping through the val data
        for i in range(len(val_batched_labels)):
            x = val_batched_text[i]
            y = val_batched_labels[i]
            preds = model(x)
            test_preds.append(preds.data.cpu().numpy())
            test_targets.append(y)
            
        test_preds = [k[i] for k in test_preds for i in range(k.shape[0])]
        test_targets = [i for item in test_targets for i in item]
        test_eval = model.eval(test_preds, test_targets)
        print('epoch:'+str(epoch)+' precision:'+str(test_eval[0])+' recall:'+
              str(test_eval[1])+' f1:'+str(test_eval[2]))
        
        #storing the details
        performance[-1]['epoch'] = epoch
        performance[-1]['loss'] = loss
        performance[-1]['train_precision'] = eval[0]
        performance[-1]['train_recall'] = eval[1]
        performance[-1]['train_f1'] = eval[2]
        performance[-1]['val_precision'] = test_eval[0]
        performance[-1]['val_recall'] = test_eval[1]
        performance[-1]['val_f1'] = test_eval[2]
        torch.save(model.state_dict(), 'models/'+out_file+'.bin')
        torch.save(bertoptimizer.state_dict(), 'optimizer/'+out_file+'.bin')
        with open('performance/'+out_file+'.pkl', 'wb') as f:
            pickle.dump(performance, f)

**MODEL-1**

*BERT-FINETUNE*

In this model, I am loading the pretrained bert-base model. I average the features of the final layer of the BERT model and pass them through a fully connected layer for classification. This model achieves an f1-score of 0.75. The bert weights are finetuned. 

In [16]:
class suggestionmodelBERTFinetune(nn.Module):
    def __init__(self):
        super(suggestionmodelBERTFinetune, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('/home/aniket/Downloads/bert-base-uncased.tar.gz').cuda()
        self.bert_model.train()
        self.linear = nn.Linear(768, 1)
        self.loss_fn = nn.BCELoss(reduce = True)
    
    def forward(self, text):
        bert_avg_representaions = []
        for instance in text:
            #preparing text to be input to BERT model
            instance = '[CLS] '+instance+' [SEP]'
            tokenized_text = self.tokenizer.tokenize(instance)
            if len(tokenized_text) > 512:
                tokenized_text = tokenized_text[:512]
            
            indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
            tokens_tensor = torch.tensor([indexed_tokens]).long().cuda()
            segment_ids = torch.zeros(tokens_tensor.size()).long().cuda()
            #the bert model return a list of size 12 (12 layers), each of size (1, seq_length, 768)
            encoder_layers, _ = self.bert_model(tokens_tensor, segment_ids)
            #getting the features of the last layer
            bert_representation = encoder_layers[-1]
            bert_representation = torch.mean(bert_representation, dim = 1)
            bert_avg_representaions.append(bert_representation)
        
        representation = torch.stack(bert_avg_representaions)
        prediction = F.sigmoid(self.linear(representation))
        return prediction
    
    def loss(self, preds, y):
        #loss calculation
        y = torch.tensor(y).float().cuda()
        preds = preds.reshape(-1)
        return self.loss_fn(preds, y)
    
    def eval(self, preds, y):
        #evaluation of the model predictions
        assert len(preds) == len(y)
        z = np.zeros(len(preds))
        for i,p in enumerate(preds):
            if p>0.5:
                z[i] = 1
        prec_score = precision_score(np.array(y), z)
        rec_score = recall_score(np.array(y), z)
        f1_score = (2 * prec_score * rec_score)/(prec_score + rec_score)
        return (prec_score, rec_score, f1_score)
        

*MODEL CREATION AND TRAINING*

In [17]:
model1 = suggestionmodelBERTFinetune().cuda()
bertoptimizer1 = BertAdam(filter(lambda p: p.requires_grad,
                                model1.parameters()), lr = 3e-5)
#training
if TRAIN:
    train(model1, bertoptimizer1, data, val_data, 'BertFinetune')
if EVAL:
    model1.load_state_dict(torch.load('models/BertFinetune.bin'))
    ModelList.append({'name':'BertFinetune', 'model':model1})

/home/aniket/.local/lib/python3.5/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


**MODEL-2**

*BERT + side-way LSTM*

This model considers all 12 layers of the bert model for classification as opposed to the previous model which considered only the last layer. The outputs of the 12 layers are passed through an lstm of sequence length 12. The batch size for this LSTM is the sequence length of the sentence. This combination of layers was inspired by https://arxiv.org/pdf/1903.05987.pdf. This paper used a weighted combination of all layers. I chose to use an LSTM to get a combined representation of all layers. I refer to this lstm as a **side-way lstm**. This model obtained an f1-score of 0.82. The bert model is fine-tuned.

In [18]:
class suggestionmodelBERTFinetuneSidewayLSTM(nn.Module):
    def __init__(self):
        super(suggestionmodelBERTFinetuneSidewayLSTM, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('/home/aniket/Downloads/bert-base-uncased.tar.gz').cuda()
        self.bert_model.train()
        self.lstm = nn.LSTM(768, 256)
        self.linear = nn.Linear(256, 1)
        self.loss_fn = nn.BCELoss(reduce = True)
    
    def forward(self, text):
        bert_avg_representaions = []
        for instance in text:
            instance = '[CLS] '+instance+' [SEP]'
            tokenized_text = self.tokenizer.tokenize(instance)
            if len(tokenized_text) > 512:
                tokenized_text = tokenized_text[:512]
            
            indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
            tokens_tensor = torch.tensor([indexed_tokens]).long().cuda()
            segment_ids = torch.zeros(tokens_tensor.size()).long().cuda()
            encoder_layers, _ = self.bert_model(tokens_tensor, segment_ids)
            bert_representation = torch.squeeze(torch.stack(encoder_layers))
            #side-way lstm
            _, (bert_lstm_representation, _) = self.lstm(bert_representation)
            #taking max across the side-way lstm representations of all words in the sentence
            bert_representation = torch.max(torch.squeeze(bert_lstm_representation), dim = 0)[0]
            bert_avg_representaions.append(bert_representation)
        
        representation = torch.stack(bert_avg_representaions)
        prediction = F.sigmoid(self.linear(representation))
        return prediction
    
    def loss(self, preds, y):
        y = torch.tensor(y).float().cuda()
        preds = preds.reshape(-1)
        entropy = preds * torch.log(preds) + (1 - preds) * torch.log(preds)
        entropy = - torch.sum(entropy)
        loss = self.loss_fn(preds, y) + 0.05 * entropy  
        return loss
    
    def eval(self, preds, y):
        assert len(preds) == len(y)
        z = np.zeros(len(preds))
        for i,p in enumerate(preds):
            if p>0.5:
                z[i] = 1
        prec_score = precision_score(np.array(y), z)
        rec_score = recall_score(np.array(y), z)
        f1_score = (2 * prec_score * rec_score)/(prec_score + rec_score)
        return (prec_score, rec_score, f1_score)

In [19]:
model2 = suggestionmodelBERTFinetuneSidewayLSTM().cuda()
bertoptimizer2 = BertAdam(filter(lambda p: p.requires_grad,
                                model2.parameters()), lr = 3e-5)
#training
if TRAIN:
    train(model2, bertoptimizer2, data, val_data, 'BertFinetuneSidewayLstm')
if EVAL:
    model2.load_state_dict(torch.load('models/BertFinetuneSidewayLstm.bin'))
    ModelList.append({'name':'BertFinetuneSidewayLstm', 'model':model2})

**MODEL-3**

*BERT + side-way lstm + 1d cnn*

This model follows from the previous model. It also uses the side-way lstm. It takes the features from all the words and passes it though 1 conv1d layers. The final representations are obtained by taking a max across all words and an average across all words and combining the former and latter vector through concat. This is the first model in which I freezed the bert model. This model gives an f1-score on 0.85. This model takes 2 minutes per epoch on a GTX-1060 and this result can be achieved in about 30 epochs of training.

In [20]:
class suggestionmodelBERTFreezedSidewayLSTMCNN(nn.Module):
    def __init__(self):
        super(suggestionmodelBERTFreezedSidewayLSTMCNN, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('/home/aniket/Downloads/bert-base-uncased.tar.gz').cuda()
        self.bert_model.eval()
        self.lstm = nn.LSTM(768, 512)
        self.cnn1d_1 = nn.Conv1d(512, 256, kernel_size = 3, padding = 1, bias = True)
        #self.cnn1d_2 = nn.Conv1d(256, 64, kernel_size = 3, padding = 1, bias = True)
        self.linear = nn.Linear(512, 1)
        #weight = torch.tensor([3.])
        self.loss_fn = nn.BCELoss(reduce = True)
    
    def forward(self, text):
        bert_avg_representaions = []
        for instance in text:
            instance = '[CLS] '+instance+' [SEP]'
            tokenized_text = self.tokenizer.tokenize(instance)
            if len(tokenized_text) > 512:
                tokenized_text = tokenized_text[:512]
            
            indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
            tokens_tensor = torch.tensor([indexed_tokens]).long().cuda()
            segment_ids = torch.zeros(tokens_tensor.size()).long().cuda()
            with torch.no_grad():
                encoder_layers, _ = self.bert_model(tokens_tensor, segment_ids)
            bert_representation = torch.squeeze(torch.stack(encoder_layers))
            _, (bert_lstm_representation, _) = self.lstm(bert_representation)
            #reshaping the data to be fit for 1d conv
            bert_lstm_representation = torch.transpose(bert_lstm_representation, 2, 1)
            #convolution
            bert_lstm_representation = F.relu(self.cnn1d_1(bert_lstm_representation))
            #bert_lstm_representation = F.relu(self.cnn1d_2(bert_lstm_representation))
            bert_lstm_representation_max = torch.squeeze(torch.max(bert_lstm_representation, dim = 2)[0])
            bert_lstm_representation_avg = torch.squeeze(torch.mean(bert_lstm_representation, 
                                                               dim = 2))
            bert_lstm_representation = torch.cat((bert_lstm_representation_avg, bert_lstm_representation_max))
            bert_avg_representaions.append(bert_lstm_representation)
        
        representation = torch.stack(bert_avg_representaions)
        prediction = F.sigmoid(self.linear(representation))
        return prediction
    
    def loss(self, preds, y):
        y = torch.tensor(y).float().cuda()
        preds = preds.reshape(-1)
        #entropy calculation, acts as a regularizer
        entropy = preds * torch.log(preds) + (1 - preds) * torch.log(preds)
        entropy = - torch.sum(entropy)
        loss = self.loss_fn(preds, y) + 0.05 * entropy  
        return loss
    
    def eval(self, preds, y):
        assert len(preds) == len(y)
        z = np.zeros(len(preds))
        for i,p in enumerate(preds):
            if p>0.5:
                z[i] = 1
        prec_score = precision_score(np.array(y), z)
        rec_score = recall_score(np.array(y), z)
        f1_score = (2 * prec_score * rec_score)/(prec_score + rec_score)
        return (prec_score, rec_score, f1_score)

In [21]:
model3 = suggestionmodelBERTFreezedSidewayLSTMCNN().cuda()
bertoptimizer3 = BertAdam(filter(lambda p: p.requires_grad,
                                model3.parameters()), lr = 3e-4)
if TRAIN:
    train(model3, bertoptimizer3, data, val_data, 'BERTFreezedSidewayLSTMCNN')
if EVAL:
    model3.load_state_dict(torch.load('models/BERTFreezedSidewayLSTMCNN.bin'))
    ModelList.append({'name':'BERTFreezedSidewayLSTMCNN', 'model':model3})

**Model-4**

*frozen bert*

This model is used to prove that the addition of side-way lstm and 1d convolutions are responsible for the rise in f1-score. This is a simple frozen bert model, with a fc layer on top. It gives an f1-score of about 0.73.

In [22]:
class suggestionmodelBERTFreezed(nn.Module):
    def __init__(self):
        super(suggestionmodelBERTFreezed, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('/home/aniket/Downloads/bert-base-uncased.tar.gz').cuda()
        self.bert_model.eval()
        self.linear = nn.Linear(768, 1)
        self.loss_fn = nn.BCELoss(reduce = True)
    
    def forward(self, text):
        bert_avg_representaions = []
        for instance in text:
            instance = '[CLS] '+instance+' [SEP]'
            tokenized_text = self.tokenizer.tokenize(instance)
            if len(tokenized_text) > 512:
                tokenized_text = tokenized_text[:512]
            
            indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
            tokens_tensor = torch.tensor([indexed_tokens]).long().cuda()
            segment_ids = torch.zeros(tokens_tensor.size()).long().cuda()
            with torch.no_grad():
                encoder_layers, _ = self.bert_model(tokens_tensor, segment_ids)
            bert_representation = encoder_layers[-1]
            bert_representation = torch.mean(bert_representation, dim = 1)
            bert_avg_representaions.append(bert_representation)
        
        representation = torch.stack(bert_avg_representaions)
        prediction = F.sigmoid(self.linear(representation))
        return prediction
    
    def loss(self, preds, y):
        y = torch.tensor(y).float().cuda()
        preds = preds.reshape(-1)
        #entropy calculation, acts as a regularizer
        entropy = preds * torch.log(preds) + (1 - preds) * torch.log(preds)
        entropy = - torch.sum(entropy)
        loss = self.loss_fn(preds, y) + 0.05 * entropy  
        return loss
    
    def eval(self, preds, y):
        assert len(preds) == len(y)
        z = np.zeros(len(preds))
        for i,p in enumerate(preds):
            if p>0.5:
                z[i] = 1
        prec_score = precision_score(np.array(y), z)
        rec_score = recall_score(np.array(y), z)
        f1_score = (2 * prec_score * rec_score)/(prec_score + rec_score)
        return (prec_score, rec_score, f1_score)

In [23]:
model4 = suggestionmodelBERTFreezed().cuda()
bertoptimizer4 = BertAdam(filter(lambda p: p.requires_grad,
                                model4.parameters()), lr = 3e-5)
if TRAIN:
    train(model4, bertoptimizer4, data, val_data, 'BertFreezed.bin')
if EVAL:
    model4.load_state_dict(torch.load('models/BertFreezed.bin'))
    ModelList.append({'name':'BertFreezed', 'model':model4})

**Model - 5**

*BERT + SIDEWAY LSTM + LSTM*

This model also uses sideway lstm. Instead of using 1d cnn across all words, here, I use an lstm across all words. the final hidden vector of the lstm is used for classification. This give an f1-score of about 0.81.

In [24]:
class suggestionmodelBERTFreezedSidewayLSTMLSTM(nn.Module):
    def __init__(self):
        super(suggestionmodelBERTFreezedSidewayLSTMLSTM, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('/home/aniket/Downloads/bert-base-uncased.tar.gz').cuda()
        self.bert_model.eval()
        self.lstm = nn.LSTM(768, 1024)
        self.lstm2 = nn.LSTM(1024, 1024)
        self.linear = nn.Linear(1024, 1)
        #weight = torch.tensor([3.])
        self.loss_fn = nn.BCELoss(reduce = True)
    
    def forward(self, text):
        bert_avg_representaions = []
        for instance in text:
            instance = '[CLS] '+instance+' [SEP]'
            tokenized_text = self.tokenizer.tokenize(instance)
            if len(tokenized_text) > 512:
                tokenized_text = tokenized_text[:512]
            
            indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
            tokens_tensor = torch.tensor([indexed_tokens]).long().cuda()
            segment_ids = torch.zeros(tokens_tensor.size()).long().cuda()
            with torch.no_grad():
                encoder_layers, _ = self.bert_model(tokens_tensor, segment_ids)
            bert_representation = torch.squeeze(torch.stack(encoder_layers))
            _, (bert_lstm_representation, _) = self.lstm(bert_representation)
            #reshaping the data to be fit for lstm
            bert_lstm_representation = torch.transpose(bert_lstm_representation, 1, 0)
            #lstm
            _,(bert_lstm_representation, _) = self.lstm2(bert_lstm_representation)
            
            bert_lstm_representation = torch.squeeze(bert_lstm_representation)
            bert_avg_representaions.append(bert_lstm_representation)
        
        representation = torch.stack(bert_avg_representaions)
        prediction = F.sigmoid(self.linear(representation))
        return prediction
    
    def loss(self, preds, y):
        y = torch.tensor(y).float().cuda()
        preds = preds.reshape(-1)
        #entropy calculation, acts as a regularizer
        entropy = preds * torch.log(preds) + (1 - preds) * torch.log(preds)
        entropy = - torch.sum(entropy)
        loss = self.loss_fn(preds, y) + 0.05 * entropy  
        return loss
    
    def eval(self, preds, y):
        assert len(preds) == len(y)
        z = np.zeros(len(preds))
        for i,p in enumerate(preds):
            if p>0.5:
                z[i] = 1
        prec_score = precision_score(np.array(y), z)
        rec_score = recall_score(np.array(y), z)
        f1_score = (2 * prec_score * rec_score)/(prec_score + rec_score)
        return (prec_score, rec_score, f1_score)

In [25]:
model5 = suggestionmodelBERTFreezedSidewayLSTMLSTM().cuda()
bertoptimizer5 = BertAdam(filter(lambda p: p.requires_grad,
                                model5.parameters()), lr = 3e-5)
if TRAIN:
    train(model5, bertoptimizer5, data, val_data, 'BERTFreezedSidewayLSTMLSTM')
if EVAL:
    model5.load_state_dict(torch.load('models/BERTFreezedSidewayLSTMLSTM.bin'))
    ModelList.append({'name':'BERTFreezedSidewayLSTMLSTM', 'model':model5})

The below block gives the f1-score for all the trained models on the validation sets.

In [26]:
#evaluation
def evaluation(val_data, model_dict):
    model = model_dict['model']
    model_name = model_dict['name']
    val_batched_text, val_batched_labels = batchify(val_data)
    #looping through the val data
    test_preds = []
    test_targets = []
    for i in range(len(val_batched_labels)):
        x = val_batched_text[i]
        y = val_batched_labels[i]
        preds = model(x)
        test_preds.append(preds.data.cpu().numpy())
        test_targets.append(y)

    test_preds = [k[i] for k in test_preds for i in range(k.shape[0])]
    test_targets = [i for item in test_targets for i in item]
    test_eval = model.eval(test_preds, test_targets)
    print('model:'+str(model_name)+' precision:'+str(test_eval[0])+' recall:'+
          str(test_eval[1])+' f1:'+str(test_eval[2]))


In [27]:
for model in ModelList:
    evaluation(val_data, model)

/home/aniket/.local/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


model:BertFinetune precision:0.8652173913043478 recall:0.6722972972972973 f1:0.7566539923954373
model:BertFinetuneSidewayLstm precision:0.8299319727891157 recall:0.8243243243243243 f1:0.8271186440677967
model:BERTFreezedSidewayLSTMCNN precision:0.753315649867374 recall:0.9594594594594594 f1:0.8439821693907875
model:BertFreezed precision:0.5914634146341463 recall:0.9831081081081081 f1:0.7385786802030455
model:BERTFreezedSidewayLSTMLSTM precision:0.825 recall:0.7804054054054054 f1:0.8020833333333333


Writing final result to csv file

In [22]:
def csvwrite(test_data, test_data_csv, model):
    test_batched_text, test_batched_labels = batchify(test_data)
    #looping through the val data
    test_preds = []
    for i in range(len(test_batched_labels)):
        x = test_batched_text[i]
        y = test_batched_labels[i]
        preds = model(x)
        test_preds.append(preds.data.cpu().numpy())
    test_preds = [k[i] for k in test_preds for i in range(k.shape[0])]
    assert len(test_preds) == len(test_data_csv)
    print(len(test_preds))
    for i, preds in enumerate(test_preds):
        if preds > 0.5:
            test_data_csv[i][2] = 1
        else:
            test_data_csv[i][2] = 0
    with open('AniketDidolkar.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows(test_data_csv)

In [23]:
csvwrite(test_data, test_data_csv, ModelList[2]['model'])

/home/aniket/.local/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


833
